In [1]:
# ipywidgets version with "update counter", render time, and a change-log panel.
# Also includes an optional Debug panel for intermediate images.

%matplotlib inline
import time, datetime as dt
import cv2, numpy as np, matplotlib.pyplot as plt
from ipywidgets import (
    HBox, VBox, IntSlider, FloatSlider, Checkbox, Dropdown, Label, Output, Layout, Button
)
from IPython.display import display, clear_output
from skimage.morphology import skeletonize, remove_small_objects

# ----------------------------
# Load image
# ----------------------------
img_path = "binary.png"   # <-- change to your file
img0 = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
if img0 is None:
    raise FileNotFoundError(f"Could not read: {img_path}")

if img0.ndim == 3:
    img_gray = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
else:
    img_gray = img0.copy()
if img_gray.dtype != np.uint8:
    img_gray = cv2.normalize(img_gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# ----------------------------
# Utilities
# ----------------------------
def binarize(gray, method="Otsu", thresh=128, invert=False, blur=0):
    g = gray
    if blur > 0:
        k = int(2*round(blur/2)+1)
        g = cv2.GaussianBlur(g, (k, k), 0)
    if method == "Otsu":
        _, bw = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    elif method == "Adaptive":
        bksz = int(thresh) | 1
        bksz = max(3, min(151, bksz))
        bw = cv2.adaptiveThreshold(g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, bksz, 2)
    else:
        _, bw = cv2.threshold(g, int(thresh), 255, cv2.THRESH_BINARY)
    if invert:
        bw = 255 - bw
    return bw

def line_kernel(length, thickness=1, angle_deg=0):
    L = max(3, int(length))
    T = max(1, int(thickness))
    size = int(np.ceil(np.sqrt(2)*L)) + 4
    canv = np.zeros((size, size), np.uint8); c = size//2
    cv2.line(canv, (c-L//2, c), (c+L//2, c), 255, T)
    M = cv2.getRotationMatrix2D((c, c), angle_deg, 1.0)
    rot = cv2.warpAffine(canv, M, (size, size), flags=cv2.INTER_NEAREST, borderValue=0)
    rot[rot > 0] = 1
    return rot

def oriented_opening(bw01, length, thickness, max_angle=8.0, step=2.0):
    angles = np.arange(-float(max_angle), float(max_angle)+1e-6, float(step))
    out = np.zeros_like(bw01, np.uint8)
    for a in angles:
        k = line_kernel(length, thickness, a)
        er = cv2.erode(bw01, k, iterations=1)
        op = cv2.dilate(er, k, iterations=1)
        out = np.maximum(out, op)
    return out

def overlay_mask_on_gray(gray, mask01, line_alpha=0.85, bg_fade=0.4, bg_to='white'):
    base = gray.astype(np.float32)
    target = 255.0 if bg_to == 'white' else 0.0
    base = (1.0 - bg_fade) * base + bg_fade * target
    base = np.clip(base, 0, 255).astype(np.uint8)
    base_bgr = cv2.cvtColor(base, cv2.COLOR_GRAY2BGR)

    color = np.zeros_like(base_bgr); color[..., 2] = 255
    m = mask01.astype(bool)
    out = base_bgr.copy()
    out[m] = (line_alpha * color[m] + (1.0 - line_alpha) * base_bgr[m]).astype(np.uint8)
    return cv2.cvtColor(out, cv2.COLOR_BGR2RGB)

# ----------------------------
# Widgets (fixed: use keyword args)
# ----------------------------
from ipywidgets import HBox, VBox, IntSlider, FloatSlider, Checkbox, Dropdown, Label, Output, Layout, Button

w_method = Dropdown(options=["Otsu", "Adaptive", "Fixed"], value="Otsu", description="Binarize")
w_thresh = IntSlider(value=128, min=0, max=255, description="Fixed/Block", continuous_update=True)
w_invert = Checkbox(value=False, description="Invert")
w_blur   = IntSlider(value=0, min=0, max=15, description="Pre-Blur", continuous_update=True)

w_len    = IntSlider(value=41, min=5,   max=151, step=2,  description="Kernel Len",   continuous_update=True)
w_thk    = IntSlider(value=1,  min=1,   max=11,  step=1,  description="Kernel Thick", continuous_update=True)
w_ang    = FloatSlider(value=8.0, min=0.0, max=20.0, step=0.5, description="±Angle",     continuous_update=True)
w_step   = FloatSlider(value=2.0, min=0.5, max=5.0,  step=0.5, description="Angle step", continuous_update=True)

w_dilate = IntSlider(value=1,  min=0,   max=6,   step=1,  description="Dilate px",   continuous_update=True)
w_area   = IntSlider(value=50, min=0,   max=500, step=10, description="Min area",    continuous_update=True)
w_skel   = Checkbox(value=True, description="Skeletonize")

w_bgfade = FloatSlider(value=0.4, min=0.0, max=1.0,  step=0.05, description="Background Fade", continuous_update=True)
w_bgto   = Dropdown(options=["white", "black"], value="white", description="Fade To")
w_alpha  = FloatSlider(value=0.9, min=0.1, max=1.0, step=0.05, description="Line Opacity",     continuous_update=True)

w_scale  = FloatSlider(value=0.9, min=0.3, max=1.5, step=0.1, description="Display scale", continuous_update=True)
w_debug  = Checkbox(value=True, description="Show Debug Panel")
btn_save = Button(description="Save overlay", button_style='')
lbl_count = Label("Updates: 0")
lbl_time  = Label("Render: -- ms")
out_plot  = Output(layout=Layout(border='1px solid #444', width='100%'))
out_log   = Output(layout=Layout(border='1px solid #444', height='140px', overflow_y='auto'))


# ----------------------------
# Change-log helper
# ----------------------------
def log(msg):
    with out_log:
        ts = dt.datetime.now().strftime("%H:%M:%S.%f")[:-3]
        print(f"[{ts}] {msg}")

# Attach logging to all controls
def attach_logging(widget, name):
    def _on_change(change):
        if change["name"] == "value":
            log(f"{name} -> {change['new']}")
            render()
    widget.observe(_on_change, names="value")

for w, n in [
    (w_method, "Binarize"), (w_thresh, "Fixed/Block"), (w_invert, "Invert"), (w_blur, "Pre-Blur"),
    (w_len, "Kernel Len"), (w_thk, "Kernel Thick"), (w_ang, "±Angle"), (w_step, "Angle step"),
    (w_dilate, "Dilate px"), (w_area, "Min area"), (w_skel, "Skeletonize"),
    (w_bgfade, "Background Fade"), (w_bgto, "Fade To"), (w_alpha, "Line Opacity"),
    (w_scale, "Display scale"), (w_debug, "Show Debug Panel")
]:
    attach_logging(w, n)

# Save button
def on_save(_):
    # Save last rendered overlay if available
    if _cache.get("overlay") is not None:
        path = "overlay_traced.png"
        cv2.imwrite(path, cv2.cvtColor(_cache["overlay"], cv2.COLOR_RGB2BGR))
        log(f"Saved: {path}")
btn_save.on_click(on_save)

# ----------------------------
# Render
# ----------------------------
_cache = {"count": 0, "overlay": None}

def render():
    t0 = time.time()
    _cache["count"] += 1

    # 1) Binarize
    bw = binarize(img_gray, method=w_method.value, thresh=w_thresh.value,
                  invert=w_invert.value, blur=w_blur.value)
    bw01 = (bw > 0).astype(np.uint8)

    # 2) Oriented opening
    opened = oriented_opening(bw01, length=w_len.value, thickness=w_thk.value,
                              max_angle=w_ang.value, step=w_step.value)

    # 3) Dilate + remove specks
    if w_dilate.value > 0:
        K = cv2.getStructuringElement(cv2.MORPH_RECT, (w_dilate.value, w_dilate.value))
        opened = cv2.dilate(opened, K, 1)
    if w_area.value > 0:
        opened_bool = opened.astype(bool)
        opened_bool = remove_small_objects(opened_bool, min_size=int(w_area.value))
        opened = opened_bool.astype(np.uint8)

    # 4) Skeleton (optional)
    traced = skeletonize(opened.astype(bool)).astype(np.uint8) if w_skel.value else opened

    # 5) Overlay
    overlay = overlay_mask_on_gray(img_gray, traced, line_alpha=w_alpha.value,
                                   bg_fade=w_bgfade.value, bg_to=w_bgto.value)
    _cache["overlay"] = overlay

    # 6) Draw
    with out_plot:
        clear_output(wait=True)
        h, w = overlay.shape[:2]
        fig_w = max(5, int(w/160))
        fig_h = max(4, int(h/160))
        fig = plt.figure(figsize=(fig_w*w_scale.value, fig_h*w_scale.value))
        if not w_debug.value:
            plt.imshow(overlay); plt.axis('off')
            plt.text(8, 18, f"Update #{_cache['count']}", color='yellow',
                     fontsize=10, bbox=dict(facecolor='black', alpha=0.4, pad=3))
        else:
            # Debug panel (2x2)
            plt.subplot(2,2,1); plt.imshow(img_gray, cmap='gray'); plt.title('Original'); plt.axis('off')
            plt.subplot(2,2,2); plt.imshow(bw, cmap='gray'); plt.title('Binarized'); plt.axis('off')
            plt.subplot(2,2,3); plt.imshow(opened, cmap='gray'); plt.title('Opened (near-horizontal)'); plt.axis('off')
            plt.subplot(2,2,4); plt.imshow(overlay); plt.title('Overlay'); plt.axis('off')
            fig.suptitle(f"Update #{_cache['count']}", fontsize=12)
        plt.show()

    # timings
    ms = int((time.time() - t0) * 1000)
    lbl_count.value = f"Updates: {_cache['count']}"
    lbl_time.value  = f"Render: {ms} ms"

# Initial draw
render()

# ----------------------------
# Layout
# ----------------------------
left_col  = VBox([
    w_method, w_thresh, w_invert, w_blur,
    w_len, w_thk, w_ang, w_step,
    w_dilate, w_area, w_skel
], layout=Layout(width='320px'))

right_col = VBox([
    w_bgfade, w_bgto, w_alpha, w_scale, w_debug,
    HBox([lbl_count, lbl_time, btn_save]),
    Label("Change Log:"), out_log
], layout=Layout(width='360px'))

ui = HBox([left_col, right_col], layout=Layout(align_items='flex-start', width='100%'))
display(ui, out_plot)


Output(layout=Layout(border_bottom='1px solid #444', border_left='1px solid #444', border_right='1px solid #44…